<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFEpostProcessingRAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Installation
---



In [1]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

Mon Jul 27 23:13:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Core(s) per socket:  1
Thread(s) per core:  2


In [3]:
%%time
# %%capture
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 165 (delta 60), reused 20 (delta 4), pack-reused 0
Receiving objects: 100% (165/165), 48.48 KiB | 591.00 KiB/s, done.
Resolving deltas: 100% (60/60), done.
PLEASE READ
********************************************************************************************************
Changes:
1. Default stable version is now 0.14.  Nightly is now 0.15.  We have fixed the long conda install.  Hooray!
2. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.15, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh s'
     

In [4]:
%%time
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
!pip install memory_profiler
# install parquet libraries
!pip install git+https://github.com/dask/fastparquet
!pip install python-snappy
!pip install pyarrow 

CPU times: user 865 ms, sys: 153 ms, total: 1.02 s
Wall time: 6min 24s


In [9]:
from pathlib import Path
import os

In [10]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box

In [11]:
import cudf
import dask_cudf

#Post-processing process 

1.   Load and concatenate parquet files into dask_cudf
2.   Count 'Source"
1.   Merge with Shapefile (gdf)
1.   Create output data









In [12]:
# Paths
pathShapefile = Path('/content/drive/My Drive/Colab Notebooks/01_Repository/FFE/data/shapefile')
pathParquets = Path('/content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison')

os.listdir(pathParquets)

['scenario320_pid23070.parquet',
 'scenario317_pid23072.parquet',
 'scenario321_pid23071.parquet',
 'scenario321_pid23073.parquet',
 'scenario319_pid23070.parquet',
 'scenario323_pid23071.parquet',
 'scenario320_pid23072.parquet',
 'scenario324_pid23071.parquet',
 'scenario322_pid23071.parquet',
 'scenario324_pid23073.parquet',
 'scenario321_pid23070.parquet',
 'scenario323_pid23073.parquet',
 'scenario322_pid23073.parquet',
 'scenario319_pid23072.parquet',
 'scenario318_pid23072.parquet',
 'scenario325_pid23073.parquet',
 'scenario323_pid23072.parquet',
 'scenario322_pid23072.parquet',
 'scenario325_pid23070.parquet',
 'scenario327_pid23073.parquet',
 'scenario326_pid23071.parquet',
 'scenario326_pid23073.parquet',
 'scenario325_pid23071.parquet',
 'scenario323_pid23070.parquet',
 'scenario324_pid23070.parquet',
 'scenario321_pid23072.parquet',
 'scenario322_pid23070.parquet',
 'scenario328_pid23073.parquet',
 'scenario325_pid23072.parquet',
 'scenario326_pid23070.parquet',
 'scenario

In [17]:
def read_and_concatenate_parquets(path=pathParquets):
  L = []
  files = pathParquets.glob('*.parquet')
  number = 0
  for file in files:
    number += 1
    print(f"file loaded : {number}")
    pqt = cudf.read_parquet(file, engine='auto')
    L.append(pqt)
  df = dask_cudf.concat(L, npartitions=4)
  return df

def countSourceAndMerge(df,nameShapefile):
  burnCount = df.Source.value_counts()
  df = pd.DataFrame({'source': burnCount.index, 'count': burnCount.values})
  #dask gdf from gdf
  gdf = gpd.read_file(pathShapefile / nameShapefile)
  cudf_gdf = cudf.DataFrame.from_pandas(gdf)
  ## need rename column for merge !!!!
  cudf_gdf.rename(columns={'FID': 'source'}, inplace=True)
  dask_gdf = dask_cudf.from_cudf(cudf_gdf, npartitions=4)
  dask_gdf = dask_gdf[['source', 'geometry']]
  
  # merge
  merged = df.merge(dask_gdf, on=['source'], how='left').compute()

  return merged

def createShapefile(df):
  gdf = gpd.GeoDataFrame(df, geometry='geometry')
  gdf.to_file(os.path.join(str(pathShapefile) + "/" + "Burn3000scenarioWellington.shp"))

In [ ]:
concatDf = read_and_concatenate_parquets()

file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario320_pid23070.parquet


/usr/local/lib/python3.6/site-packages/cudf/io/parquet.py:196: UserWarning: Using CPU via PyArrow to read Parquet dataset.
  warnings.warn("Using CPU via PyArrow to read Parquet dataset.")


file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario317_pid23072.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario321_pid23071.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario321_pid23073.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario319_pid23070.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario323_pid23071.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario320_pid23072.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario324_pid23071.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario322_pid23071.parquet
file loaded : /content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison/scenario324_pid23073.parquet
f

In [ ]:
eCu = cudf.read_parquet(p / 'Copy of edge_data.parquet')
ePd = pd.read_parquet(p / 'Copy of edge_data.parquet')

In [ ]:
type(eCu), type(ePd)

(cudf.core.dataframe.DataFrame, pandas.core.frame.DataFrame)

In [ ]:
eCu.columns

Index(['source', 'target', 'distance', 'bearing', 'IgnProb_bl'], dtype='object')

In [ ]:
%time
test_eCu = eCu['IgnProb_bl'] * 100 / 4.252

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 16.9 µs


In [ ]:
%time
test_ePd = ePd['IgnProb_bl'] * 100 / 4.252

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


In [ ]:
eCuDask = dask_cudf.from_cudf(eCu, npartitions=2)

In [ ]:
%time
test_eCuDask = eCuDask['IgnProb_bl'] * 100 / 4.252

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs


In [ ]:
%time
Concat_ePd = pd.concat([ePd,ePd])

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 51.3 µs


In [ ]:
%time
Concat_eCu = cudf.concat([eCu,eCu])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [ ]:
%time
Concat_eCuDask = dask_cudf.concat([eCuDask, eCuDask]).compute()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [ ]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

Thu Jul  9 21:00:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    43W / 250W |   1789MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------